## Accelerate Inference: Neural Network Pruning

In [14]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.3.0


In [26]:
from google.colab import drive
drive.mount('/content/drive')

!ls '/content/drive/My Drive/Colab Notebooks/10605-hw5_datasets/dataset'
file_path = '/content/drive/My Drive/Colab Notebooks/10605-hw5_datasets/dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
train_images.pkl  train_labels.pkl  val_images.pkl  val_labels.pkl


In [27]:
# untar
# !tar -xvzf dataset.tar.gz

# load train
train_images = pickle.load(open(file_path+ 'train_images.pkl', 'rb'))
train_labels = pickle.load(open(file_path+ 'train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open(file_path+ 'val_images.pkl', 'rb'))
val_labels = pickle.load(open(file_path+ 'val_labels.pkl', 'rb'))

tar (child): dataset.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [28]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [29]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 23, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 23, 23, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 11, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        1

In [134]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50
703/703 [==============================] - 3s 4ms/step - loss: 1.2466 - accuracy: 0.6587 - val_loss: 1.2289 - val_accuracy: 0.6772
Epoch 2/50
703/703 [==============================] - 3s 4ms/step - loss: 1.2335 - accuracy: 0.6707 - val_loss: 1.2340 - val_accuracy: 0.6693
Epoch 3/50
703/703 [==============================] - 3s 4ms/step - loss: 1.2294 - accuracy: 0.6747 - val_loss: 1.2298 - val_accuracy: 0.6737
Epoch 4/50
703/703 [==============================] - 3s 4ms/step - loss: 1.2221 - accuracy: 0.6831 - val_loss: 1.2301 - val_accuracy: 0.6721
Epoch 5/50
703/703 [==============================] - 3s 4ms/step - loss: 1.2226 - accuracy: 0.6807 - val_loss: 1.2260 - val_accuracy: 0.6745
Epoch 6/50
703/703 [==============================] - 3s 4ms/step - loss: 1.2176 - accuracy: 0.6880 - val_loss: 1.2157 - val_accuracy: 0.6891
Epoch 7/50
703/703 [==============================] - 3s 4ms/step - loss: 1.2170 - accuracy: 0.6870 - val_loss: 1.2143 - val_accuracy: 0.6875
Epoch 

In [135]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 4ms/step - loss: 1.2063 - accuracy: 0.6950


In [146]:
# perform pruning here

# get the weights 
weights = model.get_weights()

import copy
weights2 = copy.deepcopy(weights)

# print(len(weights))
# print('weights:')
# for i in range(len(weights)):
#   print(weights[i].shape)

# you can use set_weights() to set some weights to zero, e.g.,
# some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf

ratio = 0.095
for i in range(0, len(weights)):
  c = weights[i].shape[-1] 
  top_s = int(np.floor(c * ratio))

  idx = np.random.choice(c, top_s, replace=False)
  # print(idx)
  if len(weights[i].shape) == 1:
    weights[i][idx] = 0
  elif len(weights[i].shape) == 4:
    weights[i][:,:,:,idx] = 0
  else:
    weights[i][:, idx] = 0


# weights[7][:10]=0
model.set_weights(weights)

# evaluate again to see how the accuracy changes
results = model.evaluate(val_images, val_labels, batch_size=128)
model.set_weights(weights2)

total_w = 0
for w in weights:
  total_w += np.prod(w.shape)

# temp = ratio*(total_w - 32 - 32 - 64 - 512 - 5) / total_w
scores = results[1] + ratio
print(scores/2)

20/20 [==============================] - 0s 3ms/step - loss: 1.2614 - accuracy: 0.6451
0.37007425785064696


In [147]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model.save_weights("my_model_weights.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>